In [1]:
import tensorflow as tf
import numpy as np 
from sklearn.datasets import fetch_california_housing
from IPython.display import clear_output, Image, display, HTML

###### Do not modify here ###### 
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = graph_def
    #strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
###### Do not modify  here ######

###### Implement Data Preprocess here ######
housing = fetch_california_housing()
print("Shape of dataset:", housing.data.shape)
m, n = housing.data.shape
print("Shape of label:", housing.target.shape)

#  add bias
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

#  feed into tensorflow datatype
X = tf.constant(housing_data_plus_bias, dtype=tf.float32)
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32)
#  Q: why reshape(-1, 1)?
#  A: Orginal dimension of Y is 1-D, change into 2-D

#  split into training and testing set
train_X = X[:-(m/10)]
test_X = X[-(m/10):]
train_Y = Y[:-(m/10)]
test_Y = Y[-(m/10):]

#  W = (X'X)^(-1)X'Y 
#  Y_hat = XW => learning W
train_X_trans = tf.transpose(train_X)
W = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(train_X_trans, train_X)), train_X_trans), train_Y)

###### Implement Data Preprocess here ######

###### Start TF session ######
with tf.Session() as sess:
    # calculate Y_hat by test_X * W
    Y_hat = tf.matmul(test_X, W)
    
    error = abs(Y_hat.eval()-test_Y.eval())/test_Y.eval()
    print(error.sum()/int(test_Y.get_shape()[0]))
    #show_graph(tf.get_default_graph().as_graph_def())
###### Start TF session ######

('Shape of dataset:', (20640, 8))
('Shape of label:', (20640,))
0.346348429835
